In [1]:
import pandas as pd
books=pd.read_csv('books_with_categories.csv')

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base",top_k=None,device=0)
classifier("I love this ")

c:\Users\KRISH\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


[[{'label': 'joy', 'score': 0.9778217077255249},
  {'label': 'anger', 'score': 0.00690877391025424},
  {'label': 'neutral', 'score': 0.004429477732628584},
  {'label': 'sadness', 'score': 0.0038780963514000177},
  {'label': 'fear', 'score': 0.0034606074914336205},
  {'label': 'disgust', 'score': 0.002587498864158988},
  {'label': 'surprise', 'score': 0.0009138151071965694}]]

In [3]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [4]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.654841423034668},
  {'label': 'neutral', 'score': 0.16985194385051727},
  {'label': 'sadness', 'score': 0.11640876531600952},
  {'label': 'surprise', 'score': 0.02070068195462227},
  {'label': 'disgust', 'score': 0.019100716337561607},
  {'label': 'joy', 'score': 0.0151612414047122},
  {'label': 'anger', 'score': 0.003935152664780617}]]

In [5]:
classifier(books["description"][0].split("."))

[[{'label': 'surprise', 'score': 0.7296032905578613},
  {'label': 'neutral', 'score': 0.14038528501987457},
  {'label': 'fear', 'score': 0.06816212832927704},
  {'label': 'joy', 'score': 0.04794234782457352},
  {'label': 'anger', 'score': 0.00915635097771883},
  {'label': 'disgust', 'score': 0.002628469606861472},
  {'label': 'sadness', 'score': 0.002122158883139491}],
 [{'label': 'neutral', 'score': 0.449370801448822},
  {'label': 'disgust', 'score': 0.27359163761138916},
  {'label': 'joy', 'score': 0.10908281058073044},
  {'label': 'sadness', 'score': 0.0936271920800209},
  {'label': 'anger', 'score': 0.04047831892967224},
  {'label': 'surprise', 'score': 0.02697017416357994},
  {'label': 'fear', 'score': 0.006879049353301525}],
 [{'label': 'neutral', 'score': 0.6462163925170898},
  {'label': 'sadness', 'score': 0.24273289740085602},
  {'label': 'disgust', 'score': 0.043422624468803406},
  {'label': 'surprise', 'score': 0.028300542384386063},
  {'label': 'joy', 'score': 0.01421148143

In [6]:
sentences = books["description"][0].split(".")
predictions=classifier(sentences)

In [7]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [9]:
predictions[3]

[{'label': 'fear', 'score': 0.928167998790741},
 {'label': 'anger', 'score': 0.03219102695584297},
 {'label': 'neutral', 'score': 0.012808704748749733},
 {'label': 'sadness', 'score': 0.008756889030337334},
 {'label': 'surprise', 'score': 0.008597930893301964},
 {'label': 'disgust', 'score': 0.008431846275925636},
 {'label': 'joy', 'score': 0.0010455820010975003}]

In [10]:
sorted(predictions[0],key=lambda x:x["label"])

[{'label': 'anger', 'score': 0.00915635097771883},
 {'label': 'disgust', 'score': 0.002628469606861472},
 {'label': 'fear', 'score': 0.06816212832927704},
 {'label': 'joy', 'score': 0.04794234782457352},
 {'label': 'neutral', 'score': 0.14038528501987457},
 {'label': 'sadness', 'score': 0.002122158883139491},
 {'label': 'surprise', 'score': 0.7296032905578613}]

In [11]:
import numpy as np
emotion_labels= [x["label"] for x in predictions[0]]
emotion_scores = {label:[] for label in emotion_labels}
isbn=[]

def calculate_max_emotion_scores(predictions):
    per_emotion_scores={label:[] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions=sorted(prediction,key = lambda x:x["label"])
        for index,label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])

        return {label:np.max(scores) for label,scores in per_emotion_scores.items()}

In [12]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences=books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores= calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])
    

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [13]:
emotion_scores

{'surprise': [0.00915635097771883,
  0.005966869182884693,
  0.041300997138023376,
  0.016036199405789375,
  0.01362438965588808,
  0.009158486500382423,
  0.003973892889916897,
  0.023656249046325684,
  0.3006698787212372,
  0.01765839010477066],
 'neutral': [0.002628469606861472,
  0.002887075301259756,
  0.024568455293774605,
  0.06069517880678177,
  0.12224282324314117,
  0.012228667736053467,
  0.0038164728321135044,
  0.009091983549296856,
  0.2794811427593231,
  0.1779269576072693],
 'fear': [0.06816212832927704,
  0.0038098874501883984,
  0.1040615662932396,
  0.001691634999588132,
  0.09504348784685135,
  0.03679506108164787,
  0.0012676806654781103,
  0.4044956862926483,
  0.08446498215198517,
  0.04945705831050873],
 'joy': [0.04794234782457352,
  0.7044203877449036,
  0.7672369480133057,
  0.1617567539215088,
  0.0083356574177742,
  0.043375857174396515,
  0.8725652098655701,
  0.01337516214698553,
  0.0013904988300055265,
  0.032197605818510056],
 'anger': [0.1403852850198

In [15]:
from tqdm import tqdm
emotion_labels= [x["label"] for x in predictions[0]]
emotion_scores = {label:[] for label in emotion_labels}
isbn=[]
for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences=books["description"][i].split(".")
    predictions=classifier(sentences)
    max_scores= calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])
    

100%|██████████| 5197/5197 [06:10<00:00, 14.02it/s]  


In [16]:
emotions_df=pd.DataFrame(emotion_scores)
emotions_df["isbn13"]=isbn
emotions_df.head()

,fear,anger,sadness,neutral,disgust,joy,surprise,isbn13
0,0.009156,0.002628,0.068162,0.047942,0.140385,0.002122,0.729603,9780002005883
1,0.005967,0.002887,0.003810,0.704420,0.217761,0.004509,0.060646,9780002261982
2,0.041301,0.024568,0.104062,0.767237,0.042176,0.010860,0.009796,9780006178736
3,0.016036,0.060695,0.001692,0.161757,0.732686,0.020988,0.006147,9780006280897
4,0.013624,0.122243,0.095043,0.008336,0.272613,0.475880,0.012260,9780006280934


In [17]:
books=pd.merge(books,emotions_df,on="isbn13")

In [18]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,title_and_subtitle,tagged_description,simple_categories,fear,anger,sadness,neutral,disgust,joy,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.009156,0.002628,0.068162,0.047942,0.140385,0.002122,0.729603
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.005967,0.002887,0.003810,0.704420,0.217761,0.004509,0.060646
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.041301,0.024568,0.104062,0.767237,0.042176,0.010860,0.009796
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.016036,0.060695,0.001692,0.161757,0.732686,0.020988,0.006147
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.013624,0.122243,0.095043,0.008336,0.272613,0.475880,0.012260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction,0.025156,0.001939,0.094667,0.002254,0.010511,0.857255,0.008218
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction,0.005602,0.003775,0.018216,0.400263,0.338892,0.005487,0.227765
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction,0.008463,0.009147,0.013295,0.620452,0.329754,0.010788,0.008101
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction,0.005475,0.034544,0.003970,0.258353,0.648011,0.017372,0.032275


In [19]:
books.to_csv("books_with_emotions.csv",index=False)